In [ ]:
import os, sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp

# modelica interface
import buildingspy.simulate.Simulator as si
import buildingspy.io.outputfile as buipy_io

# thermal
import  CoolProp.CoolProp as CP
from CoolProp.HumidAirProp import HAPropsSI
import psychrolib as psylib
psylib.SetUnitSystem(psylib.SI)

In [ ]:
sys.path.insert(0, os.path.join('.','..','calibration'))
from thermodynamics import *
from postprocessing import *
from plotting import *
from calibration_MgCl2_class import *

# import code of mathieu
import media_model_mgcl2 as model_mg
import media_model_water as model_w

In [ ]:
%load_ext autoreload
# only reload the function imported by "aimport"
%autoreload 1
%aimport thermodynamics, postprocessing, plotting, calibration_MgCl2_class

# Data preprocessing

In [ ]:
path_expdata = os.path.join(os.path.expanduser('~'), 'GIT', 'TheGreeFa_EnBA', 'Data_input', 'measurement')

## Chen2016

In [ ]:
df_chen_data = pd.read_excel(os.path.join(path_expdata, "actual_data_converted/Chen2016.xlsx"), index_col=0)

In [ ]:
df_chen_data['x_d_in_equ'] = np.vectorize(calc_x_des_equ)(df_chen_data.T_d_in, df_chen_data.xi_d_in)
df_chen_data["RH_a_in"] = np.vectorize(get_RH)(df_chen_data.T_a_in, df_chen_data.x_a_in) *100
df_chen_data["RH_a_o_exp"] = np.vectorize(get_RH)(df_chen_data.T_a_o_exp, df_chen_data.x_a_o_exp) *100
df_chen_data["RH_d_in_equ"] = np.vectorize(get_RH)(df_chen_data.T_d_in, df_chen_data.x_d_in_equ) *100

In [ ]:
plot_with_des_point_df(df_data=df_chen_data, chart_template='minimal', use_line_highlight = False, ncol = 3, ) # save_fig_name="Chen_minimal.png"

## Dempav

In [ ]:
df_exp_dempav = pd.read_excel(os.path.join(path_expdata, "actual_data_converted/dempav_T_des_corrected.xlsx"), index_col=0)
df_exp_dempav.head()

In [ ]:
df_exp_dempav['x_d_in_equ'] = np.vectorize(calc_x_des_equ)(df_exp_dempav.T_d_in, df_exp_dempav.xi_d_in)
df_exp_dempav["RH_a_in"] = df_exp_dempav["RH_a_in"] * 100
df_exp_dempav["RH_a_o_exp"] = df_exp_dempav["RH_a_o_exp"] * 100
df_exp_dempav["RH_d_in_equ"] = np.vectorize(get_RH)(df_exp_dempav.T_d_in, df_exp_dempav.x_d_in_equ) *100

In [ ]:
plot_air_state_df(df_data=df_exp_dempav, chart_template="minimal", alpha=0.8, )  # save_fig_name='dempav_all.png'

In [ ]:
df_exp_dempav.query('T_a_in - T_a_o_exp > 2').shape

In [ ]:
plot_with_des_point_df(df_data=df_exp_dempav.query('T_a_in - T_a_o_exp > 2'), chart_template="minimal", alpha=0.8, save_fig_name='dempav_cooling.png') # save_fig_name='dempav_cooling.png' 

# Calculate Nu/Sh through calibration
- the main code for this section can be found in calibration_MgCl2_class.py

## Chen2016

In [ ]:
def run_calib_LiCl(prefix_output, file_name):
    bib_path = os.path.join(os.path.expanduser("~"),"GIT", 'TheGreeFa_EnBA')
    model_path = 'BrineGrid.Fluid.Absorbers.Examples.Validation.'
    
    # run chen2016
    model_name = 'Calibration_Chen_2016'
    print("*****start {}******".format(model_name))

    df = pd.read_excel(os.path.join(bib_path, "Data_input", "measurement", "actual_data_converted/{}".format(file_name)),\
                       index_col=0)
    print(df)

    # generate object for calibration class
    
    LiCl_obj = Calibration_H_M_transfer(t_stop=3600, exchange_area=450*1.5*0.75*0.75, d_e=0.01, type_desiccant='LiCl',\
        cross_area=1.5*0.75, void_fraction = 0.9,bib_path=bib_path, model_path = model_path, nNodes=8, mNodes=8, \
            model_name=model_name, prefix_outputfolder=prefix_output, flow_config = 'cross')

    output_path = LiCl_obj.get_output_folder()

    # # run multiple calibration
    LiCl_obj.mp_run(df=df, num_core=mp.cpu_count()-2)
    # # run single calibration
    # LiCl_obj.run_calibration(input_dataset=df.iloc[9,:])

    df_output = LiCl_obj.read_results(path=output_path)
    df_output.to_csv(path_or_buf=os.path.join(output_path, 'final_output.csv'))

    print('{} finished'.format(model_name))
    return

In [ ]:
run_calib_LiCl("New_Git_", "Chen2016.xlsx")

## Dempav

In [ ]:
def run_calib_MgCl2_dempav(prefix_output, file_name):
    # # run MgCl2
    model_name = 'Calibration_MgCl2_dempav'
    print("*****start {}******".format(model_name))

    bib_path = os.path.join(os.path.expanduser("~"),"GIT",'TheGreeFa_EnBA')
    model_path = 'BrineGrid.Fluid.Absorbers.Examples.Validation.'

    df = pd.read_excel(os.path.join(bib_path, "Data_input", "measurement", "actual_data_converted/{}").format(file_name),index_col = 0) .query('T_a_in - T_a_o_exp > 2')
    print(df)
    
    # # generate object for calibration class
    MgCl2_obj = Calibration_H_M_transfer(t_stop=3600, exchange_area=6.5, d_e=0.04, cross_area = 0.065, \
        void_fraction = 0.95, delta_T = 0.5, bib_path=bib_path, model_path = model_path, \
            nNodes=8, mNodes=8, model_name=model_name, prefix_outputfolder=prefix_output)
    
    output_path = MgCl2_obj.get_output_folder()
    print("output path is: {}.".format(output_path))
    
    try:
        MgCl2_obj.run_calibration(input_dataset=df.iloc[8,:]) # single simulation
        # MgCl2_obj.mp_run(df=df, num_core=8) #mp.cpu_count()- 4
    except:
        print('Calibration for {} + {} didn\'t completely finished.'.format(model_name, file_name))

    df_output = MgCl2_obj.read_results(path=output_path)
    df_output.to_csv(path_or_buf=os.path.join(output_path, 'final_output.csv'))

    print('{} (dempav) finished'.format(model_name))
    return

In [ ]:
run_calib_MgCl2_dempav("2021_oct_dempav_3_", "dempav_T_des_corrected.xlsx")

# Get Nu/Sh formula through regression
- the main code for this section can be found in calibration_MgCl2_class.py

- Note: the parameter a-f calculated through regression should be kept as small as possbile (-2 to +2 recommended) to avoid the deviation caused by expoential calculation

## Chen2016

In [10]:
# check results of last step:
bath_path = os.path.join(os.path.expanduser("~"),"GIT","TheGreeFa_EnBA", "Results", "modelica")
folder_name = "2021_oct_Calibration_Chen_2016"
file_name = "final_output.csv"

df = pd.read_csv(os.path.join(bath_path, folder_name, file_name), index_col=0)
df

,T_a_in,x_a_in,m_a_in,T_d_in,xi_d_in,m_d_in,T_a_o_exp,x_a_o_exp,T_d_o_exp,xi_d_o_exp,...,T_d_o_sim_new,xi_d_o_sim_new,xi_d_out,x_d_in_equ,Re,Pr,Sc,epsilon_w,epsilon_T,epsilon_h
1.0,298.75,0.0162,1.85,288.05,0.23,2.61,292.35,0.0091,294.45,0.2288,...,293.301880,0.228865,0.228848,0.006411,996.698754,0.723822,0.665818,0.725270,0.598131,0.687494
10.0,297.15,0.0152,1.94,289.95,0.2961,2.82,293.25,0.0075,296.25,0.2945,...,295.042755,0.294639,0.294540,0.005028,1048.989749,0.722987,0.665865,0.757000,0.541667,0.710071
11.0,297.35,0.016,1.98,290.85,0.3007,2.85,294.05,0.0077,297.05,0.299,...,296.498871,0.298972,0.298976,0.005172,1070.471776,0.723793,0.665906,0.766523,0.507692,0.716903
12.0,298.25,0.0152,1.94,291.25,0.3052,2.9,294.45,0.0075,297.25,0.3036,...,296.439911,0.303645,0.303636,0.005146,1046.027445,0.722846,0.665787,0.765884,0.542857,0.717881
13.0,298.35,0.0143,1.95,290.95,0.3182,2.85,294.25,0.0069,297.05,0.3166,...,296.253540,0.316602,0.316597,0.004590,1050.698532,0.721900,0.665719,0.762118,0.554054,0.714170
14.0,299.05,0.0159,1.99,292.35,0.3215,2.86,295.75,0.0077,298.95,0.3196,...,298.045074,0.319695,0.319676,0.004912,1071.149531,0.723472,0.665776,0.746246,0.492537,0.697101
15.0,298.75,0.0143,1.93,291.55,0.3312,2.88,294.75,0.0066,297.65,0.3295,...,296.987885,0.329501,0.329500,0.004322,1038.855687,0.721849,0.665691,0.771678,0.555556,0.723933
2.0,301.75,0.0202,1.87,291.35,0.24,2.63,296.05,0.0116,298.35,0.2385,...,297.330353,0.238568,0.238541,0.007597,1001.761856,0.727614,0.665828,0.682360,0.548077,0.650017
3.0,300.65,0.0181,1.92,290.85,0.255,2.65,295.05,0.0099,297.45,0.2535,...,296.837555,0.253523,0.253494,0.006824,1030.361472,0.725560,0.665793,0.727236,0.571429,0.687884
4.0,303.15,0.0177,1.89,292.05,0.26,2.66,296.65,0.0097,298.95,0.2585,...,298.072235,0.258555,0.258530,0.007183,1007.699590,0.724833,0.665567,0.760675,0.585586,0.709571


In [ ]:
# declare object
obj_postproc = EnBA_M_post_processing(folder_name = "2021_oct_Calibration_Chen_2016", file_name = "final_output.csv")

# run regression
obj_postproc.run_regression_generate_results(method="scipy", Nu_num_training=12, \
                                            Nu_if_sorted=True, Sh_num_training=12, \
                                            Sh_if_sorted=False, physical_Nu_Sh=False, if_plot=True, max_iter=10000)

## Dempav

In [15]:
# check results of last step:
bath_path = os.path.join(os.path.expanduser("~"),"GIT","TheGreeFa_EnBA", "Results", "modelica")
folder_name = "2021_oct_dempav_Calibration_MgCl2_dempav"
file_name = "final_output.csv"

df = pd.read_csv(os.path.join(bath_path, folder_name, file_name), index_col=0)
df

,T_a_in,RH_a_in,m_a_in,T_d_in,xi_d_in,m_d_in,T_a_o_exp,RH_a_o_exp,T_d_o_exp,x_a_in,...,T_d_o_sim_new,xi_d_o_sim_new,xi_d_out,x_d_in_equ,Re,Pr,Sc,epsilon_w,epsilon_T,epsilon_h
0,298.02,0.480462,0.018848,287.44,0.318302,0.102670,290.21,0.476947,288.662602,0.009457,...,288.687988,0.318079,0.318087,0.004515,665.090372,0.716941,0.616288,0.745485,0.738185,0.742920
1,300.51,0.553303,0.036283,292.20,0.318186,0.102670,294.81,0.538242,294.359825,0.012683,...,294.646210,0.317649,0.317741,0.006170,1274.153030,0.719955,0.616900,0.609066,0.685921,0.636076
14,289.91,0.884298,0.044123,283.27,0.276833,0.104691,286.61,0.603531,285.632948,0.010582,...,285.351837,0.276326,0.276275,0.004276,1591.385499,0.719162,0.614945,0.758945,0.496988,0.681611
17,290.31,0.519762,0.018728,276.01,0.277934,0.104689,285.55,0.388650,276.796881,0.006337,...,276.794373,0.277791,0.277792,0.002552,673.443779,0.714750,0.614671,0.757589,0.332867,0.502555
18,291.66,0.473850,0.018106,282.41,0.276896,0.104689,287.64,0.468699,282.901939,0.006289,...,282.898682,0.276824,0.276824,0.004029,648.722721,0.714521,0.614921,0.656881,0.434595,0.519292
19,297.19,0.421433,0.043522,284.42,0.323335,0.049907,292.08,0.474445,287.391565,0.007873,...,287.477417,0.322917,0.322939,0.003572,1537.918140,0.715421,0.616040,0.327043,0.400157,0.367827
2,298.02,0.458659,0.032644,284.16,0.318572,0.102670,287.98,0.506137,286.568520,0.009022,...,286.562714,0.318190,0.318196,0.003614,1151.701330,0.716494,0.616260,0.686702,0.724387,0.706884
20,298.77,0.357802,0.044014,283.13,0.317486,0.049666,289.96,0.478044,287.616638,0.007340,...,287.629883,0.317013,0.317024,0.003390,1548.556024,0.714674,0.616280,0.416577,0.563299,0.507790
21,299.94,0.344053,0.050034,284.36,0.311315,0.049425,291.08,0.485803,289.168386,0.007566,...,289.157227,0.310883,0.310890,0.003825,1755.236470,0.714759,0.616496,0.360665,0.568678,0.491942
22,298.60,0.478863,0.037139,286.42,0.303124,0.049105,292.23,0.520544,290.147268,0.009762,...,290.146759,0.302516,0.302531,0.004607,1308.774193,0.717181,0.616406,0.502859,0.522989,0.513834


In [ ]:
# declare object
obj_postproc = EnBA_M_post_processing(folder_name = "2021_oct_dempav_Calibration_MgCl2_dempav", file_name = "final_output.csv")

# run regression
obj_postproc.run_regression_generate_results(method="scipy", Nu_num_training=12, \
                                               Nu_if_sorted=False, Sh_num_training=12, \
                                               Sh_if_sorted=False, physical_Nu_Sh=False, if_plot=True, max_iter=10000)

# run simulation with new Nu-Sh Formular (calculated by regression)
- Firstly, the formular needs to be passed into modelica manually

In [ ]:
def run_Chen_fomular_NuSh(prefix_output, file_name):
    # run simulation using implemented formular for Nu&Sh in Modelica #
    model_name = 'Validation_new_formular_Chen_2016'
    print("*****start {}******".format(model_name))

    bib_path = os.path.join(os.path.expanduser("~"),"GIT",'TheGreeFa_EnBA')
    model_path = 'BrineGrid.Fluid.Absorbers.Examples.Validation.'

    df = pd.read_excel(os.path.join(bib_path, "Data_input", "measurement", "actual_data_converted/{}".format(file_name)),\
                       index_col = 0)
    print(df)

    # # generate object for calibration class
    LiCl2_obj = Calibration_H_M_transfer(t_stop=1800, exchange_area=450*1.5*0.75*0.75, type_desiccant='LiCl',\
        d_e=0.01, cross_area = 1.5*0.75, void_fraction = 0.9, bib_path=bib_path, model_path = model_path, flow_config = 'cross', \
        nNodes=10, mNodes=10, model_name=model_name, prefix_outputfolder=prefix_output, if_Nu_Sh_const=False)

    output_path = LiCl2_obj.get_output_folder()
    
    print("output path is: {}.".format(output_path))
    
    # LiCl2_obj.run_single_simulation(input_dataset=df.iloc[0,:])
    LiCl2_obj.mp_run(df=df, mode="simulation", num_core=mp.cpu_count()-2)

    df_output = LiCl2_obj.read_results(path=output_path, mode="simulation")
    df_output.to_csv(path_or_buf=os.path.join(output_path, 'final_output.csv'))

    print('{} finished'.format(model_name))
    return

In [ ]:
# run_Chen_fomular_NuSh(prefix_output='Chen_regress_fomular_Nu_Sh_2021Nov_2_', file_name='Chen2016.xlsx')

# Validate the results

## Chen2016

### read simulation results

In [ ]:
bath_path = os.path.join(os.path.expanduser("~"),"GIT","TheGreeFa_EnBA", "Results", "modelica")
folder_name = "Chen_regress_fomular_Nu_Sh_2021Nov_2_Calibration_Chen_2016"
file_name = "final_output.csv"

df_sim_results = pd.read_csv(os.path.join(bath_path, folder_name, file_name), index_col=0)
df_sim_results

### Model results verification by comparing with expdata and plotting

In [ ]:
df_sim_results["delta_T_a_exp"] =  df_sim_results.T_a_in - df_sim_results.T_a_o_exp
df_sim_results["delta_x_a_exp"] =  df_sim_results.x_a_in - df_sim_results.x_a_o_exp

In [ ]:
df_sim_results["delta_T_a_sim"] =  df_sim_results.T_a_in - df_sim_results.T_a_o_sim_reg
df_sim_results["delta_x_a_sim"] =  df_sim_results.x_a_in - df_sim_results.x_a_o_sim_reg

In [ ]:
df_results_compare = df_sim_results[["delta_T_a_exp", "delta_x_a_exp", "delta_T_a_sim", "delta_x_a_sim"]]

In [ ]:
plot_compare_2_cols(df_results_compare.delta_T_a_exp, df_results_compare.delta_T_a_sim, deviation = 0.1, x_label = "Änderung der Lufttemp. (Messung)", y_label = "Änderung der Lufttemp. (Simulation)", save_fig="Chen_T")

In [ ]:
plot_compare_2_cols(df_results_compare.delta_x_a_exp, df_results_compare.delta_x_a_sim, deviation=0.1, x_label = "Änderung der Luftfeuchte (Messung)", y_label = "Änderung der Luftfeuchte (Simulation)", save_fig="Chen_x")

## Dempav

### read sim results

In [ ]:
bath_path = os.path.join(os.path.expanduser("~"),"GIT","TheGreeFa_EnBA", "Results", "modelica")
folder_name = "Dempav_regress_fomular_Nu_Sh_2021Nov_Validation_MgCl2_dempav"
file_name = "final_output.csv"

df_sim_results = pd.read_csv(os.path.join(bath_path, folder_name, file_name), index_col=0)
df_sim_results

In [ ]:
df_sim_results["delta_T_a_exp"] =  df_sim_results.T_a_in - df_sim_results.T_a_o_exp
df_sim_results["delta_x_a_exp"] =  df_sim_results.x_a_in - df_sim_results.x_a_o_exp
df_sim_results["delta_T_a_sim"] =  df_sim_results.T_a_in - df_sim_results.T_a_o_sim_reg
df_sim_results["delta_x_a_sim"] =  df_sim_results.x_a_in - df_sim_results.x_a_o_sim_reg

In [ ]:
df_results_compare = df_sim_results[["delta_T_a_exp", "delta_x_a_exp", "delta_T_a_sim", "delta_x_a_sim"]]

In [ ]:
plot_compare_2_cols(df_results_compare.delta_T_a_exp, df_results_compare.delta_T_a_sim, deviation = 0.2, x_label = "Änderung der Lufttemp. (Messung)", y_label = "Änderung der Lufttemp. (Simulation)", save_fig="dempav_T")

In [ ]:
plot_compare_2_cols(df_results_compare.delta_x_a_exp, df_results_compare.delta_x_a_sim, deviation=0.2, x_label = "Änderung der Luftfeuchte (Messung)", y_label = "Änderung der Luftfeuchte (Simulation)", save_fig="dempav_x")